# Modelo Preditivo de Fraude - Marcador de POC (DRAFT)

## Objetivo
Os POCs reportados são focados em agências com maior ocorrência de fraude, não constituindo, portanto, um registro completo dos POCs ocorridos. Esta análise tem por objetivo  a melhorar a performance do modelo preditivo, a partir da marcação dos POCs de forma mais completa, por meio da análise dos eventos ocorridos nas agências/ATMs com POCs reportados.

## Libraries

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import HiveContext 
hive_context = HiveContext(sc)
# para plots
%matplotlib inline

In [2]:
#import pandas as pd
#from pyspark.sql import SQLContext
#print sc
#df = pd.DataFrame([("foo", 1), ("bar", 2)], columns=("k", "v"))
#print type(df)
#print df
#sqlCtx = SQLContext(sc)
#sqlCtx.createDataFrame(df).show()

## Carga das tabelas

In [3]:
# Load files from HDFS into Dataframe
# Load files from HDFS into Dataframe

rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')
poc_in = '/user/x181472/export/BASE_POC_ATM_090816.csv'
tranlog_in = hive_context.table('default.ext_juntatudo12')


In [4]:
#from pyspark.sql.functions import dayofmonth, month, concat, lit, year

#rdd_eventlog = rdd_eventlog.select('nr_sequ_even', 'dh_even', 'cd_eqpt', 'nr_idef_secu',  
#                               concat(dayofmonth(rdd_eventlog['dh_even']), lit("/"), 
#                               month(rdd_eventlog['dh_even']),lit("/"), year(rdd_eventlog['dh_even']))).\
#                               groupBy('nr_idef_secu', 'cd_eqpt', 'nr_sequ_even','dh_even').count()

In [5]:
rdd_eventlog

DataFrame[nr_sequ_pont_grup_eqpt: int, nr_sequ_even: int, dh_even: string, cd_eqpt: string, nr_idef_secu: string]

In [6]:
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))



Número de linhas carregadas: 18341921


## Definição de funções

In [7]:
# funcao para converter data de string para date e zerar hora, minutos e segundos
#OBS: alteração abaixo para considerar segundo com ponto decimal (evitar erro de formatação)
#funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'), DateType())
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

In [8]:
#datetime.strptime('2016-02-14 02:08:34.0', '%Y-%m-%d %H:%M:%S.%f').replace(minute=0, hour=0, second=0, microsecond=0)
#type(d)
#d.replace(minute=0, hour=0, second=0, microsecond=0)

#datetime.date(d.year, d.month, d.day)

## Carga de dados brutos
Os dados brutos de log das ATMs e o de ocorrência de POCs carregados e descritos abaixo

### Dados de log de eventos

In [9]:
df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



In [10]:
#df_eventlog.limit(10).toPandas()

In [11]:
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

In [12]:
# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')


In [13]:
#df_eventlog.limit(10).orderBy(col("nr_eventos").desc()).toPandas()

In [14]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

df_eventlog.printSchema()   

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [50]:
#df_eventlog.limit(10).orderBy(col("nr_eventos").desc()).toPandas()

In [51]:
#print ("\nNúmero de linhas carregadas: " + str("%d" % (df_eventlog.count())))
#print ("\nNúmero de linhas após retirar NAs: " + str("%d" % (df_eventlog.dropna().count())))

In [15]:
# DEPOIS: acertar abaixo para pegar direto ultima data do arquivo original como data_ref
# Filtrando últimos 90 dias
dias_a_subtrair = 90 
import datetime, time
from dateutil.relativedelta import relativedelta

# obtendo faixa do período de histórico a partir da data de referencia
data_ref = datetime.datetime(2016, 7, 16) 
noventa_dias_antes = data_ref + relativedelta(days=-dias_a_subtrair)
print (noventa_dias_antes)
type (noventa_dias_antes)

2016-04-17 00:00:00


datetime.datetime

In [16]:
data_ref = ("2016-04-17")

dt_sql = (time.mktime(datetime.datetime.strptime(data_ref, "%Y-%m-%d").timetuple()))
dt_sql

1460862000.0

In [17]:
# FILTRANDO SOMENTE ULTIMOS 90 DIAS
df_eventlog = df_eventlog.where("CAST(dh_even AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
#df_eventlog.show()

## Obtendo Eventos Marcadores de POC
Usando como baseline o arquivo de POCs reportados, serão identificados os eventos fora do padrão das agências/ATMs para serem usados como marcadores de POC no modelo preditivo.

In [18]:
# calculando percentis de baseline
df_eventlog_percen = df_eventlog.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [19]:
df_eventlog_percen.take(5)

[Row(id=u'105-172-7023', nr_eventos=2),
 Row(id=u'105-173-6778', nr_eventos=1),
 Row(id=u'105-173-3017', nr_eventos=1),
 Row(id=u'105-172-5743', nr_eventos=1),
 Row(id=u'105-173-5535', nr_eventos=2)]

In [20]:
percentil_min = 25
percentil_max = 75
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], percentil_min), np.percentile(x[1], percentil_max)))

In [21]:
rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

In [22]:
df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'percentil_min',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

In [23]:
df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- percentil_min: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [24]:
# colocar no log as colunas de percentis por agencia+atm+evento 
df_eventlog2 = df_eventlog \
    .join(df_eventlog_percen, (df_eventlog['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_eventlog['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (df_eventlog['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

In [25]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda

In [26]:
df_eventlog2.orderBy('nr_sequ_even').limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos  percentil_min  \
0          105     168         1080 2016-05-25           1            1.0   
1          105     168         1080 2016-06-02           1            1.0   
2         3618     167         1440 2016-06-17           1            NaN   
3         1530     162         1440 2016-06-10           1            1.0   
4         4212     155         1440 2016-08-15           1            1.0   
5         3409     163         1440 2016-05-30           2            1.0   
6         3322     155         1440 2016-06-10           2            1.0   
7         3350     180         1440 2016-06-21           3            1.5   
8         4212     155         1440 2016-06-16           2            1.0   
9         1227     164         1440 2016-08-13           1            1.0   

   percentil_max  
0            1.0  
1            1.0  
2            NaN  
3            1.0  
4            1.0  
5            1.0  
6            1.0  
7            2.5  
8            1.0  
9            1.0

### Dados de registro de POC
Obtém os POCs de Retenção de Cartão, somente para os últimos 90 dias.
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

Obtém somente os POCs de Golpe de **RETENTOR DE CARTÃO** inicialmente.
Identifica quais eventos que aconteceram nas agências e na data que saíram da faixa de percentil. estes eventos serão os **Marcadores de POC**.

In [27]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))

header = rdd_poc.first()
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(",")) \
    .map(lambda p: (p[0],p[2],p[4], p[8])) \
    .toDF(['data_cad', 
            'atm',
            'agencia',
            'golpe'
        ])

df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

df_poc = df_poc \
    .withColumn('atm', df_poc["atm"].cast(StringType()))\
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
df_poc = df_poc.where("golpe = 'GOLPE RETENTOR DE CARTAO' and \
            CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy('agencia', 'data_cad')
#df_poc = df_poc.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy('agencia', 'data_cad')



Número de linhas carregadas: 1019


In [28]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_poc = df_poc\
    .withColumn('atm', trim(df_poc.atm))\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0', '')) # removendo zero `esquerda


#df_poc.limit(10).toPandas()

In [29]:
df_poc.printSchema()

root
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [30]:
df_eventlog2.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- percentil_min: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [31]:
# DESNECESSARIA A PESQUISA ABAIXO AGORA

#df_eventlog2.registerTempTable("tb_logeven")
#df_poc.registerTempTable("tb_pocagen")

#df_evenpoc = sqlContext.sql("select * from tb_logeven where exists \
#                        (select agencia from tb_pocagen where tb_logeven.nr_idef_secu = tb_pocagen.agencia)")

# TESTAR AQUI SE SELECIONOU SOMENTE AGENCIAS DO POC
# tem que trazer igual ou menor ao numero de agencias do df_poc
#df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

## Selecionando Eventos por ATM/Agência em dias POC

In [32]:
# selecionar pokevento
# 1. selecionar somente datas de POC para as agencias de POC acima com inner join de poc por data
#df_x = df_evenpoc \
#    .join(df_poc, (df_poc['data_cad'] == df_evenpoc['dh_even']) &
#                (df_poc['agencia'] == df_evenpoc['nr_idef_secu']), 'inner')
df_poceven = df_eventlog2 \
    .join(df_poc, (df_poc['data_cad'] == df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'inner')

In [33]:
# filtrar somente eventos fora do percentil para agencias com POC
# a partir destes eventos filtrar conforme já fez abaixo para marcar os
# POCs para todos os eventos escolhidos maior que o percentil

#df_pokevento = df_evenpoc.filter(col("nr_eventos") > col('percentil_max'))
#df_evenpoc.filter(col("nr_eventos") > col('percentil_max'))\
#    .groupBy("nr_sequ_even")
#    .count().orderBy("count").collect()
pd_poceven = df_poceven.filter(col("nr_eventos") > col('percentil_max'))\
    .groupBy("nr_sequ_even")\
    .count().orderBy("nr_sequ_even").toPandas()
pd_poceven

nr_sequ_even  count
0           1473      1
1           1478      1
2           1568     12
3           1574      4
4           1575      3
5           1578      7
6           1579     33
7           1581      7
8           1582     17
9           1584      2
10          1585      3
11          1587     11
12          1588     13
13          1590     54
14          1599      1
15          1601      1
16          1606      8
17          1609      2
18          1615     20
19          1616     22
20          1617     34
21          1618      6
22          1619      1
23          1620     34
24          1621     15
25          1626      1
26          1627      1
27          1628     78
28          1641      1
29          1642      1
..           ...    ...
257          702     29
258         7023    133
259          718     47
260          723     35
261          724     73
262          798      2
263          799      9
264          800      6
265          801      6
266          802      5
267          803      2
268          804      2
269          806      1
270          807     16
271          808     12
272          821      1
273          853      2
274          855      1
275          883     17
276          896     11
277          897      8
278          898      6
279          899      4
280          900      1
281          902      1
282          916     40
283          928     27
284          929     23
285          933     11
286          934     38

[287 rows x 2 columns]

In [34]:
#  colocar esta lista na selecao abaixo (tirando as que nao sao relevantes))
l_evenpoc = pd_poceven.nr_sequ_even.tolist()
#l_evenpoc
# abauixo, checar se marcou eventos POC como label 1.0 e nao os demais eventos que nao sao poc



In [35]:
# aqui gerar lista de age-atm-evento acima do prercentil que ocorrem em dia poc
pd_pocAgAtEv = df_poceven.filter(col("nr_eventos") > col('percentil_max'))\
    .groupBy(["nr_idef_secu","cd_eqpt","nr_sequ_even"])\
    .count().orderBy(["nr_idef_secu","cd_eqpt","nr_sequ_even"]).toPandas()
#pd_pocAgAtEv

## Selecionando Eventos por ATM/Agência Fora de dias POC

In [36]:
# idem acima para dias nao poc para join depois, mas somente para eventos acima
#df_poceven.filter(col("nr_eventos") > col('percentil_max')).limit(10).toPandas()
df_NAOpoceven = df_eventlog2 \
    .join(df_poc, (df_poc['data_cad'] != df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'inner')
#df_NAOpoceven.count()

In [37]:
# selecionando somente eventos que ocorreram no poc
df_NAOpoceven = df_NAOpoceven\
    .filter((col("nr_sequ_even").isin(l_evenpoc)))
#df_NAOpoceven.count()

In [38]:
pd_NAOpocAgAtEv = df_NAOpoceven.filter(col("nr_eventos") > col('percentil_max'))\
    .groupBy(["nr_idef_secu","cd_eqpt","nr_sequ_even"])\
    .count().orderBy(["count"]).toPandas()
#pd_NAOpocAgAtEv

In [39]:
# 1. selecionar somente agatev que nao ocorrem fora de dia poc
merged = pd.merge(pd_NAOpocAgAtEv, pd_pocAgAtEv, left_on=['nr_idef_secu','cd_eqpt', 'nr_sequ_even' ] , 
         right_on=['nr_idef_secu','cd_eqpt', 'nr_sequ_even'], how='left')
#merged[merged['count_y'] == '']
#pd.isnull(merged.count_y)

In [40]:

# dataframe pandas somente com agencia+atm+evento que ocorrem somente em dia POC
# criar dataframe de CandidatosPOC para marcar no tabelao de features label para modelo
pd_candidatoPOC = merged[pd.isnull(merged.count_y)]
df_canddatoPOC = sqlCtx.createDataFrame(pd_candidatoPOC)
#df_canddatoPOC.toPandas()

nr_idef_secu cd_eqpt nr_sequ_even  count_x  count_y
0              220     152         5498        1      NaN
1              568     159         1653        1      NaN
2              568     151         3405        1      NaN
3              568     161         5495        1      NaN
4              568     152         7023        1      NaN
5              568     162         1473        1      NaN
6              568     151         1585        1      NaN
7              568     162         1860        1      NaN
8              568     162         4269        1      NaN
9              568     151         1642        1      NaN
10             568     164         3408        1      NaN
11             568     161         3952        1      NaN
12             568     163         1650        1      NaN
13             568     162         1609        1      NaN
14             568     162         3011        1      NaN
15             568     151         3014        1      NaN
16             568     156         5413        1      NaN
17             568     163         1619        1      NaN
18             568     164         1892        1      NaN
19             568     156          933        1      NaN
20             568     162          934        1      NaN
21             568     159          689        1      NaN
22             568     163         5401        1      NaN
23             568     151         1581        1      NaN
24             568     156         1973        1      NaN
25             568     156         6428        1      NaN
26             568     151         1473        1      NaN
27             568     163         5755        1      NaN
28             568     162         3814        1      NaN
29             568     164         1579        1      NaN
...            ...     ...          ...      ...      ...
44632         1206   10001         3283      103      NaN
44633         2093   10001         3283      103      NaN
44634          245   10001         3283      104      NaN
44635         3282     167         7023      104      NaN
44636         1217     155         7023      107      NaN
44637         2286   10001         5412      109      NaN
44638         3282   10001          569      109      NaN
44639         2082     154         3009      110      NaN
44640          270     155         1856      113      NaN
44641         3544     163          808      114      NaN
44642          112     154         1856      115      NaN
44643          344     163          934      117      NaN
44644          140     173         3283      117      NaN
44645          270     161         1856      120      NaN
44646         3282   10001         4615      120      NaN
44647         2082     168         7023      120      NaN
44648         2082     173         1856      122      NaN
44649         2082     154         5412      132      NaN
44650         3282     151         7023      133      NaN
44651         2082     173         7023      138      NaN
44652         3282   10001         3283      149      NaN
44653         4550   10001         3288      152      NaN
44654         2082     166         7023      156      NaN
44655         2082     159         7023      162      NaN
44656         2082     159         1856      172      NaN
44657         2082     192         1856      178      NaN
44658         2082     158         7023      194      NaN
44659         4550   10001         3283      196      NaN
44660         2082     182         5412      196      NaN
44661         2082     200         1856      240      NaN

[44662 rows x 5 columns]

In [41]:
# checar quantas das agencias daqui aparecem no arquivo de POC reportado

# USAR ESTE DATAFRAME PARA MARGCAR OS POCS
df_canddatoPOC = df_canddatoPOC\
   .drop(df_canddatoPOC["count_y"])\
    .withColumnRenamed('count_x','nr_eventosPOC')

In [42]:
df_canddatoPOC.count()
#df_canddatoPOC.limit(10).toPandas()

44662

In [152]:
pd_grppoccand = df_canddatoPOC.groupBy('nr_idef_secu').count().toPandas()

In [153]:
pd_grppoc = df_poc.groupBy('agencia').count().toPandas()

In [180]:
## Checagem entre POCS apontados por evento e a tabela de POC reportados 
# OK: os pocs marcados nos eventos aparecem somente em agencias reportadas como POC
# assim confirmamos que estamos pegando somente agencias com POC reportados
# mas com granularidade de ATM, que não tinhamos antes

pd_chkPOC = pd.merge(pd_grppoc, pd_grppoccand, left_on=['agencia'] , 
         right_on=['nr_idef_secu'], how='left')
pd_chkPOC

agencia  count_x nr_idef_secu  count_y
0     4268        2         4268    440.0
1     3262        1         3262   1137.0
2      787        1          787    635.0
3      670        1          670    793.0
4      220        2          220    570.0
5      065        1          NaN      NaN
6      112        2          112    888.0
7      564        1          564    703.0
8      568        1          568    768.0
9      118        1          118    492.0
10    3544        1         3544   1586.0
11     458        1          458    373.0
12     344        1          344   1148.0
13    4212        1         4212   1557.0
14    1206        1         1206    603.0
15    4550        2         4550    369.0
16    3165        1         3165    775.0
17    1320        1         1320    426.0
18    2219        1         2219    559.0
19    3282        2         3282   1032.0
20    1053        1         1053    437.0
21    1217        1         1217    525.0
22     245        1          245    544.0
23    2286        1         2286   1114.0
24    3063        1         3063    590.0
25    2175        1         2175    407.0
26    3724        1         3724   1155.0
27    2226        1         2226    444.0
28    1227        1         1227   1552.0
29    1561        1         1561    312.0
30    4688        1         4688    398.0
31    4237        1         4237    922.0
32     140        1          140   2130.0
33    1610        1         1610    339.0
34     144        1          144    696.0
35     803        1          803    591.0
36    3733        1         3733    885.0
37    4355        1         4355    771.0
38    3350        1         3350   2405.0
39    1074        2         1074    460.0
40    4406        1         4406    440.0
41     651        1          651    458.0
42     656        1          656    706.0
43     206        1          206    668.0
44    2082        2         2082   4373.0
45    1690        1         1690    907.0
46     549        1          549   1000.0
47     270        1          270    699.0
48    4256        1         4256    504.0
49     212        1          212    559.0
50    2093        1         2093    797.0
51    3146        1         3146    979.0
52    4263        1         4263    612.0

In [161]:
#merged.sort_values(by=['count_x'], ascending=[False])
# 2. para tentar pegar pocs nao reportados, tentar pegar count nao poc = 1

## Preparando dados processados
Marcando como POC as entradas do log para os eventos marcadores 

In [162]:

#df_eventlog2 = df_eventlog2 \
#   .withColumn("label",
#        when((col("nr_eventos") > col("percentil_max")) & \
#             (col("nr_sequ_even").isin(l_evenpoc))
#             ,1.0) \
#        .otherwise(0.0))
# marcação por evento poc maior que percentil ok!!

In [163]:
#df_eventlog2.filter(col("label") == 1.0).take(10)
#df_eventlog2.filter((col("nr_sequ_even") == '1599')).toPandas() # filtar por data atm e agencia tb

In [157]:
#df_eventlog2.orderBy(col('label').desc()).limit(10).toPandas()

# TESTAR ABAIXO PARA USAR LISTA DE EVENTOS POC
#maturity_udf = udf(lambda age: "adult" if age >=18 else "child", StringType())
# OU maturity_udf = udf(lambda evento, nr_eventos, percentil_max: \
# 1.0 if evento in ["1610","2082","2226", "4237", "4263", "4268"] and nr_eventos > percentil_max else 0.0 and , DoubleType())

#df = sqlContext.createDataFrame([{'name': 'Alice', 'age': 1}])
#df.withColumn("maturity", maturity_udf(df.age))

In [64]:
# FALTA: mudar media por percentil 50%
# obter as colunas de med diária por ag/equip/evento
# media
df_event_med = df_eventlog2\
    .groupBy(['nr_idef_secu','cd_eqpt','nr_sequ_even'])\
    .avg('nr_eventos').withColumnRenamed('avg(nr_eventos)', 'med')
#df_event_med.limit(10).toPandas()

In [65]:
# ver se troca de df_x aqui está correta
# concatenar dados de features calculadas com dados reais agrupados por ag/equip/evento
df_feat = df_eventlog2 \
    .join(df_event_med, (df_eventlog2['nr_idef_secu'] == df_event_med['nr_idef_secu']) &
                      (df_eventlog2['cd_eqpt'] == df_event_med['cd_eqpt']) &
                      (df_eventlog2['nr_sequ_even'] == df_event_med['nr_sequ_even']), 'left') \
    .drop(df_event_med["nr_idef_secu"]).drop(df_event_med["cd_eqpt"]).drop(df_event_med["nr_sequ_even"])

In [66]:
# criar coluna de indice de proximidade da media (iproxmed)
# calculado como número de ocorrências real diaria/média diaria, por agencia e atm
df_feat = df_feat\
    .withColumn('iproxmed', col("nr_eventos")/col("med"))
df_feat.limit(10).toPandas()

Py4JJavaError: An error occurred while calling o1087.javaToPython.
: java.lang.NullPointerException
	at org.apache.spark.sql.execution.Limit.sortBasedShuffleOn(basicOperators.scala:202)
	at org.apache.spark.sql.execution.Limit.doExecute(basicOperators.scala:210)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$5.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$5.apply(SparkPlan.scala:138)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:138)
	at org.apache.spark.sql.SQLContext$QueryExecution.toRdd$lzycompute(SQLContext.scala:933)
	at org.apache.spark.sql.SQLContext$QueryExecution.toRdd(SQLContext.scala:933)
	at org.apache.spark.sql.DataFrame.javaToPython(DataFrame.scala:1583)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.refle

In [46]:
# transpondo tipo de evento para gerar features
df_feat = df_feat\
    .drop(df_feat["med"])

In [47]:
#df_feat.groupBy(['nr_idef_secu']).count().toPandas()

In [48]:
# Antes de fazer o pivot, selecionar no log somente os eventos da lista dos pocs (l_evenpoc)
# filtrando somente eventos que serão usados no mllib (para não dar falta de memoria ao converter pandas 
# para dataframe spark abaixo)
#l_evenfeat = ["4428", "4268", "1640","3262", "787","670", "3265", "3310","1206", "3982"]
df_feat = df_feat.filter(col("nr_sequ_even").isin(l_evenpoc))

In [63]:
# OBS: selecionar somente os eventos usados no mllib como features para usar menos memoria:
pd_pivot = df_feat\
    .select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()
pd_pivot

Py4JJavaError: An error occurred while calling o1053.javaToPython.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
TungstenExchange hashpartitioning(nr_idef_secu#45,cd_eqpt#43,nr_sequ_even#24)
 TungstenProject [regexp_replace(trim(nr_idef_secu#26),^0,) AS nr_idef_secu#45,trim(cd_eqpt#25) AS cd_eqpt#43,nr_sequ_even#24,dh_even#27,nr_eventos#28,percentil_min#41,percentil_max#42]
  SortMergeOuterJoin [nr_idef_secu#26,cd_eqpt#25,nr_sequ_even#24], [nr_idef_secu#38,cd_eqpt#39,nr_sequ_even#40], LeftOuter, None
   TungstenSort [nr_idef_secu#26 ASC,cd_eqpt#25 ASC,nr_sequ_even#24 ASC], false, 0
    TungstenExchange hashpartitioning(nr_idef_secu#26,cd_eqpt#25,nr_sequ_even#24)
     TungstenSort [dh_even#27 ASC], true, 0
      ConvertToUnsafe
       Exchange rangepartitioning(dh_even#27 ASC)
        ConvertToSafe
         TungstenProject [nr_idef_secu#19 AS nr_idef_secu#26,cd_eqpt#18 AS cd_eqpt#25,cast(nr_sequ_even#16L as string) AS nr_sequ_even#24,cast(dh_even#20 as

In [52]:
# transpondo evento por tipo OK! 
# OBS: a funcao .reset_index abaixo é necessário para evitar a mesclagem das células fixas
pd_feat = pd.pivot_table(pd_pivot,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()
# acertando nomes das colunas geradas em linhas diferentes
pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]

In [53]:
pd_feat

nr_idef_secu cd_eqpt    dh_even  1473  1478  1568  1574  1575  1578  \
0               105   10001 2016-04-17   0.0   0.0   0.0   0.0   0.0   0.0   
1               105   10001 2016-04-18   0.0   0.0   0.0   0.0   0.0   0.0   
2               105   10001 2016-04-19   0.0   0.0   0.0   0.0   0.0   0.0   
3               105   10001 2016-04-20   0.0   0.0   0.0   0.0   0.0   0.0   
4               105   10001 2016-04-21   0.0   0.0   0.0   0.0   0.0   0.0   
5               105   10001 2016-04-22   0.0   0.0   0.0   0.0   0.0   0.0   
6               105   10001 2016-04-23   0.0   0.0   0.0   0.0   0.0   0.0   
7               105   10001 2016-04-24   0.0   0.0   0.0   0.0   0.0   0.0   
8               105   10001 2016-04-25   0.0   0.0   0.0   0.0   0.0   0.0   
9               105   10001 2016-04-26   0.0   0.0   0.0   0.0   0.0   0.0   
10              105   10001 2016-04-27   0.0   0.0   0.0   0.0   0.0   0.0   
11              105   10001 2016-04-28   0.0   0.0   0.0   0.0   0.0   0.0   
12              105   10001 2016-04-29   0.0   0.0   0.0   0.0   0.0   0.0   
13              105   10001 2016-04-30   0.0   0.0   0.0   0.0   0.0   0.0   
14              105   10001 2016-05-01   0.0   0.0   0.0   0.0   0.0   0.0   
15              105   10001 2016-05-02   0.0   0.0   0.0   0.0   0.0   0.0   
16              105   10001 2016-05-03   0.0   0.0   0.0   0.0   0.0   0.0   
17              105   10001 2016-05-04   0.0   0.0   0.0   0.0   0.0   0.0   
18              105   10001 2016-05-05   0.0   0.0   0.0   0.0   0.0   0.0   
19              105   10001 2016-05-06   0.0   0.0   0.0   0.0   0.0   0.0   
20              105   10001 2016-05-07   0.0   0.0   0.0   0.0   0.0   0.0   
21              105   10001 2016-05-08   0.0   0.0   0.0   0.0   0.0   0.0   
22              105   10001 2016-05-09   0.0   0.0   0.0   0.0   0.0   0.0   
23              105   10001 2016-05-10   0.0   0.0   0.0   0.0   0.0   0.0   
24              105   10001 2016-05-11   0.0   0.0   0.0   0.0   0.0   0.0   
25              105   10001 2016-05-12   0.0   0.0   0.0   0.0   0.0   0.0   
26              105   10001 2016-05-13   0.0   0.0   0.0   0.0   0.0   0.0   
27              105   10001 2016-05-14   0.0   0.0   0.0   0.0   0.0   1.0   
28              105   10001 2016-05-15   0.0   0.0   0.0   0.0   0.0   0.0   
29              105   10001 2016-05-16   0.0   0.0   0.0   0.0   0.0   0.0   
...             ...     ...        ...   ...   ...   ...   ...   ...   ...   
143302          990     153 2016-07-21   0.0   0.0   2.0   1.0   0.0   1.0   
143303          990     153 2016-07-22   0.0   0.0   0.0   0.0   0.0   0.0   
143304          990     153 2016-07-23   0.0   0.0   0.0   0.0   0.0   0.0   
143305          990     153 2016-07-24   0.0   0.0   0.0   0.0   0.0   1.0   
143306          990     153 2016-07-25   0.0   0.0   0.0   0.0   0.0   2.0   
143307          990     153 2016-07-26   0.0   0.0   1.0   0.0   0.0   1.0   
143308          990     153 2016-07-27   0.0   0.0   0.0   0.0   0.0   1.0   
143309          990     153 2016-07-28   1.0   0.0   0.0   0.0   0.0   0.0   
143310          990     153 2016-07-29   0.0   0.0   0.0   0.0   0.0   0.0   
143311          990     153 2016-07-30   0.0   0.0   0.0   0.0   0.0   2.0   
143312          990     153 2016-07-31   0.0   0.0   0.0   0.0   0.0   0.0   
143313          990     153 2016-08-01   0.0   0.0   0.0   1.0   0.0   0.0   
143314          990     153 2016-08-02   0.0   0.0   0.0   0.0   0.0   1.0   
143315          990     153 2016-08-03   0.0   0.0   0.0   0.0   0.0   1.0   
143316          990     153 2016-08-04   0.0   0.0   1.0   1.0   0.0   0.0   
143317          990     153 2016-08-05   0.0   0.0   0.0   0.0   0.0   4.0   
143318          990     153 2016-08-06   0.0   0.0   0.0   0.0   0.0   0.0   
143319          990     153 2016-08-07   0.0   0.0   0.0   0.0   0.0   0.0   
143320          990     153 2016-08-08   0.0   0.0   0.0   0.0   0.0   0.0   
143321          990     153 2

In [54]:
#c = pd_feat.columns.tolist
#c[1]

In [55]:
# criar coluna com data como string para posterior transformação para dataframe
pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat

nr_idef_secu cd_eqpt  1473  1478  1568  1574  1575  1578      1579  \
0               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
1               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
2               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
3               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
4               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
5               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
6               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
7               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
8               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
9               105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
10              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
11              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
12              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
13              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
14              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
15              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
16              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
17              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
18              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
19              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
20              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
21              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
22              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
23              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
24              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
25              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
26              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
27              105   10001   0.0   0.0   0.0   0.0   0.0   1.0  0.000000   
28              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
29              105   10001   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
...             ...     ...   ...   ...   ...   ...   ...   ...       ...   
143302          990     153   0.0   0.0   2.0   1.0   0.0   1.0  0.000000   
143303          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143304          990     153   0.0   0.0   0.0   0.0   0.0   0.0  1.529412   
143305          990     153   0.0   0.0   0.0   0.0   0.0   1.0  0.000000   
143306          990     153   0.0   0.0   0.0   0.0   0.0   2.0  0.000000   
143307          990     153   0.0   0.0   1.0   0.0   0.0   1.0  0.000000   
143308          990     153   0.0   0.0   0.0   0.0   0.0   1.0  1.529412   
143309          990     153   1.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143310          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143311          990     153   0.0   0.0   0.0   0.0   0.0   2.0  0.764706   
143312          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143313          990     153   0.0   0.0   0.0   1.0   0.0   0.0  0.000000   
143314          990     153   0.0   0.0   0.0   0.0   0.0   1.0  0.000000   
143315          990     153   0.0   0.0   0.0   0.0   0.0   1.0  0.000000   
143316          990     153   0.0   0.0   1.0   1.0   0.0   0.0  0.000000   
143317          990     153   0.0   0.0   0.0   0.0   0.0   4.0  0.000000   
143318          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.764706   
143319          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143320          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143321          990     153   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   
143

In [62]:
df_feat_alt = sqlCtx.createDataFrame(pd_feat)
# FALTA converter data em timedate de novo (converter para string no Pandas antes)

Py4JJavaError: An error occurred while calling o1044.defaultParallelism.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext
	at org.apache.spark.SparkContext.org$apache$spark$SparkContext$$assertNotStopped(SparkContext.scala:104)
	at org.apache.spark.SparkContext.defaultParallelism(SparkContext.scala:2063)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:207)
	at 

In [61]:
df_feat_alt.take(5)

Py4JJavaError: An error occurred while calling o1042.javaToPython.
: java.lang.NullPointerException
	at org.apache.spark.sql.execution.Limit.sortBasedShuffleOn(basicOperators.scala:202)
	at org.apache.spark.sql.execution.Limit.doExecute(basicOperators.scala:210)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$5.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$5.apply(SparkPlan.scala:138)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:138)
	at org.apache.spark.sql.SQLContext$QueryExecution.toRdd$lzycompute(SQLContext.scala:933)
	at org.apache.spark.sql.SQLContext$QueryExecution.toRdd(SQLContext.scala:933)
	at org.apache.spark.sql.DataFrame.javaToPython(DataFrame.scala:1583)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.refle

In [58]:
df_feat_alt.limit(10).toPandas()

Py4JJavaError: An error occurred while calling o1039.javaToPython.
: java.lang.NullPointerException
	at org.apache.spark.sql.execution.Limit.sortBasedShuffleOn(basicOperators.scala:202)
	at org.apache.spark.sql.execution.Limit.doExecute(basicOperators.scala:210)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$5.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$5.apply(SparkPlan.scala:138)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:138)
	at org.apache.spark.sql.SQLContext$QueryExecution.toRdd$lzycompute(SQLContext.scala:933)
	at org.apache.spark.sql.SQLContext$QueryExecution.toRdd(SQLContext.scala:933)
	at org.apache.spark.sql.DataFrame.javaToPython(DataFrame.scala:1583)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.refle

In [178]:
# salva tabelao para acerto abaixo
#pd_feat.to_csv(tabelao)

# alternativa
#import csv, io 
# python 2 use StringIO

#def list_to_csv_str(x):
#    """Given a list of strings, returns a properly-csv-formatted string."""
#    output = io.StringIO("")
#    csv.writer(output).writerow(x)
#    return output.getvalue().strip() # remove extra newline

# ... do stuff with your rdd ...
#rdd_feat = pd_feat.rdd.map(list_to_csv_str)
#rdd_feat.saveAsTextFile(tabelao)

In [179]:
# OBS: se já tiver o tabelao_alt gerado, pode rodar a partir daqui!!!!!
#############################################################################
# AQUI TIVE QUE FAZER ESTA GAMBIARRA: salvar como csv e alguma manipulaçao (acertar cabeçalho) para depois ler
# até resolver o problema de converter em RDD
#rdd_feat_alt = sc.textFile(tabelao_alt)

In [180]:
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_feat_alt.count())))

In [181]:
#header = rdd_feat_alt.first()
#schemaString = header
#fields = [StructField(field_name, StringType(), False) for field_name in schemaString.split(',')]
#schema = StructType(fields)

In [182]:
#rdd_feat_alt = rdd_feat_alt \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(","))

In [183]:
#df_feat_alt = sqlContext.createDataFrame(rdd_feat_alt, schema)

In [184]:
df_tidy = df_feat_alt\
    .withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))\
    .withColumn('label', df_feat_alt["label"].cast(DoubleType()))

In [185]:
df_tidy.limit(10).toPandas()

nr_idef_secu cd_eqpt  label  4268  data_even
0         2014     157    0.0   1.0 2016-07-21
1         2082     161    0.0   1.0 2016-05-23
2         2082     162    0.0   1.0 2016-06-08
3         2082     162    0.0   1.0 2016-06-10
4         2082     163    0.0   1.0 2016-05-23
5         2082     169    0.0   1.0 2016-05-23
6         2082     173    0.0   0.8 2016-05-23
7         2082     173    0.0   0.8 2016-06-23
8         2082     173    0.0   0.8 2016-08-13
9         2082     175    0.0   1.0 2016-05-13

In [186]:
df_tidy.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- label: double (nullable = true)
 |-- 4268: double (nullable = true)
 |-- data_even: timestamp (nullable = true)



In [187]:
# salvando tabelao com label
#df_tidy.toPandas().to_csv(tabelao_final)

## Modelagem
O modelo usado será o de Random Forest, implementando pipelines da library ML do Spark.
A partir da amostra de dados será selecionado um subset com 75% dos dados para treinar o modelo. 

### Treino do Modelo
Prepara a amostra de acordo com o formato de entrada de label e vetor de features esperados pelo modelo.

In [188]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [189]:
# Transformar em double as features (features selecionadas)
df_tidy_dbl = df_tidy\
    .withColumn('4268', df_tidy["4268"].cast(DoubleType()))
#    .withColumn('2915', df_tidy["2915"].cast(DoubleType()))\
#    .withColumn('3672', df_tidy["3672"].cast(DoubleType()))
#    .withColumn('1617', df_tidy["1617"].cast(DoubleType()))\
#    .withColumn('1689', df_tidy["1689"].cast(DoubleType()))\
#    .withColumn('2433', df_tidy["2433"].cast(DoubleType()))\
#    .withColumn('1746', df_tidy["1746"].cast(DoubleType()))\
#    .withColumn('1568', df_tidy["1568"].cast(DoubleType()))\
#    .withColumn('883', df_tidy["883"].cast(DoubleType()))\
#    .withColumn('2163', df_tidy["2163"].cast(DoubleType()))
#    .withColumn('3672', df_tidy["3672"].cast(DoubleType()))
    # 2915, 2267, 3672, 1617, 2568, 2433, 1746, 883, 1616

In [191]:
# rodando para as features ocm mais relevância já analisadas
assemblerTrain = VectorAssembler(
    inputCols=["4268"],
   outputCol="features")
#               ,"1617", "1689","2433", "1746", "1568","883", "2163"],
 

In [192]:
df_tmp = assemblerTrain.transform(df_tidy_dbl)

In [193]:
df_tidy_dt = df_tmp.select("label", "features")
df_tidy_dt.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [204]:
# Split the data into training and test sets (30% held out for testing)
(trainingData_rf, testData_rf) = df_tidy_dt.randomSplit([0.7, 0.3])

In [205]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_dt)
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(trainingData_rf)

In [206]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_dt)
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(trainingData_rf)

Separa a amostra de forma aleatória em dados de treino (70% da amostra) e de teste (30% da amostra, para posterior avaliação do modelo).

In [207]:
# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
#dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [208]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [209]:
#pipeline = Pipeline(stages=[rf])
# Search through decision tree's maxDepth parameter for best model
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

numFolds = 5

In [210]:
evaluator = BinaryClassificationEvaluator()

In [211]:
crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=numFolds)

In [212]:
model = crossval.fit(trainingData_rf)

Py4JJavaError: An error occurred while calling o3690.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 10 in stage 1202.0 failed 4 times, most recent failure: Lost task 10.3 in stage 1202.0 (TID 48689, srvbdrprlbr04.santanderbr.corp): java.lang.ArrayIndexOutOfBoundsException: 1
	at org.apache.spark.mllib.linalg.DenseVector.apply(Vectors.scala:560)
	at org.apache.spark.ml.evaluation.BinaryClassificationEvaluator$$anonfun$1.apply(BinaryClassificationEvaluator.scala:78)
	at org.apache.spark.ml.evaluation.BinaryClassificationEvaluator$$anonfun$1.apply(BinaryClassificationEvaluator.scala:77)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:328)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:211)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:73)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:73)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(S

In [ ]:
# Fetch best model
tree_model = model.bestModel.stages[2]
print tree_model

In [121]:
# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData_rf)

## Persistência do Modelo
Salva o modelo para posterior uso. O código comentado abaixo deve ser descomentado nas manutenções para testar se modelo foi corretamente salvo.

In [122]:
# SAVE MODEL FOR FUTURE USE (TEST IF SAVED OK)
#model.save(sc, savedModel) # Erro. Funciona com mllib. Pesquisar como salvar pipeline

### Avaliação do Modelo
Avalia o modelo usando os dados separados para teste, obtendo o erro de teste (1 - acurácia), a área sob a curva ROC (AUC) e a taxa Precision/Recall.

In [123]:
predictions = model.transform(testData_rf)

In [124]:
evaluator.explainParams()

'labelCol: label column name (default: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)'

In [125]:
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [126]:
print("AUC = %g " % (auroc))

AUC = 0.990487 


In [127]:
precrec = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

In [128]:
print("Precision/Recall = %g " % (precrec))

Precision/Recall = 0.969611 


In [129]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.00951349 


Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

In [130]:
poc_predicted = predictions.select("prediction", "indexedLabel", "features", "probability").toPandas()

In [134]:
# 6. dividir dados em treino e teste
# selecionando sample estratificada
df_tidy_dbl.groupBy("label").count().orderBy("label").show()
# N: 0.9995   S: 0.000499
# Balanceamento com marcação anterior de report de POC
# N: 0.0110104 S: 0.98898
# Balanceamento com marcação por evento fora do percentil de POC
# N:  S: 

+-----+-----+
|label|count|
+-----+-----+
|  0.0|92452|
|  1.0|27169|
+-----+-----+



In [136]:
df_tidy_dbl.groupBy(["nr_idef_secu", "label"]).count().orderBy("nr_idef_secu").show()

+------------+-----+-----+
|nr_idef_secu|label|count|
+------------+-----+-----+
|         105|  0.0| 2154|
|         105|  1.0|  561|
|        1053|  1.0|  152|
|        1053|  0.0|  416|
|        1074|  0.0|  435|
|        1074|  1.0|  106|
|         112|  1.0|  266|
|         112|  0.0|  861|
|         117|  1.0|  185|
|         117|  0.0|  650|
|         118|  1.0|  123|
|         118|  0.0|  469|
|        1206|  0.0|  531|
|        1206|  1.0|  180|
|        1207|  1.0|  108|
|        1207|  0.0|  324|
|        1217|  1.0|  136|
|        1217|  0.0|  488|
|        1227|  0.0| 1325|
|        1227|  1.0|  450|
+------------+-----+-----+
only showing top 20 rows



In [87]:
# salving the resulting dataframe
#type(poc_predicted)
poc_predicted.to_csv(previsao)
#df.write.csv('mycsv.csv')

In [88]:
# estratégia
# 1. obter a média diária da agência + atm (trocar para mediana) para cada evento
# 2. do log, somar o número de ocorrências por agencia + atm + evento agrupado por dia
# 3. montar tabelao inicial e criar coluna count/media para ver índice de desvio da média (FEATURE) por agencia/atm
# 4. criar coluna LABEL concatenando com arquivo POC por agencia + data cad (atm nao confiavel no arquivo)
# obs: desvio > 1 indica percentual acima da média e vice-versa
# 5. fazer o pivot como acima

# trocar media por mediana (FALTA)
# usar mediana para gerar o tabelao (ou percentil 75% ou soma contante) (FALTA)
# dividir pelo count para ter a feature percentil/count (FALTA)
# transposicao (filtrar tabelao por agencia + atm e join dois a dois (em loop?))
# filtrar os 3 dias após a data de poc para tirar efeitos de setup   
# 

In [137]:
# matriz de confusão
predictions.orderBy(col('prediction').desc()).show()

+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|  1.0|[0.0,2.8481012658...|         1.0|[0.0,2.8481012658...|[0.27154038634314...|[0.01357701931715...|       1.0|
|  1.0|[0.0,1.6688741721...|         1.0|[0.0,1.6688741721...|[1.02626087562955...|[0.05131304378147...|       1.0|
|  1.0|[0.0,2.2784810126...|         1.0|[0.0,2.2784810126...|[0.27154038634314...|[0.01357701931715...|       1.0|
|  1.0|[0.0,1.8507462686...|         1.0|[0.0,1.8507462686...|[0.42404962135054...|[0.02120248106752...|       1.0|
|  1.0|[0.0,2.3134328358...|         1.0|[0.0,2.3134328358...|[0.27154038634314...|[0.01357701931715...|       1.0|
|  1.0|[0.0,1.7088607594...|         1.0|[0.0,1.7088607594...|[1.0262608

## confusion matrix

In [138]:
def confusion_matrix(prevCol, labelCol):
    if prevCol == 1.0 and labelCol == 1.0:
        return 'TP'
    else:
        if prevCol == 1.0 and labelCol == 0.0:
            return 'FP'
        else:
            if prevCol == 0.0 and labelCol == 0.0:
                return 'TN'
            else:
                if prevCol == 0.0 and labelCol == 1.0:
                    return 'FN'
            
createNewColFromTwo = udf(confusion_matrix, StringType())
df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'], predictions['label']))

In [139]:
# matriz de confuao
df_rates = df_conf_matr.groupBy("conf_matr").count()
# precision
#my_precision = df_rates[df_rates['conf_matr'] == 'TP'] / (df_rates[df_rates['conf_matr'] == 'TP'] + 
#                                           df_rates[df_rates['conf_matr'] == 'FP'])
# recall
#my_recall = df_rates[df_rates['conf_matr'] == 'TP'] / (df_rates[df_rates['conf_matr'] == 'TP'] + 
#                                           df_rates[df_rates['conf_matr'] == 'FN'])
#df_saldo_explr = df_saldo_cheque.where(
#    (col('cgrp_ctbil') == '7') &
#    (col('csgrp_ctbil')   == '5'))

In [140]:
# Sensibility - habilidade do modelo classificar corretamente um POC
# Probability of being test positive when POC present.
# TP / (TP + FN)
sensitivity = 
# Specificity - habilidade do modelo classificar corretamente a ausência de POC 
#Probability of being test negative when POC absent.
# TN / (TN + FP)

SyntaxError: invalid syntax (<ipython-input-140-1d0fa326e7f0>, line 4)

In [141]:
# TESTES
predictions.filter((col("prediction") == 0.0) & (col("label") == 1.0)).count()

704

In [142]:
df_rates.collect()

[Row(conf_matr=u'TN', count=26955),
 Row(conf_matr=u'TP', count=7468),
 Row(conf_matr=u'FN', count=705),
 Row(conf_matr=u'FP', count=646)]

In [143]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 1.0)).count()

7443

In [144]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 0.0)).count()

621

In [107]:
predictions.filter((col("prediction") == 0.0) & (col("label") == 0.0)).count()

19698

In [145]:
# checagem do marcador de POC contra POCs reportados
# somente POC retencao de cartão, D-90
# trazer número de POCs para a chave agencia
# encotradas 52 agências com POC
df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
df_p.toPandas()
# agora comparar com o arquivo gerado com label

agencia  count
0      065      1
1     1053      1
2     1074      2
3      112      2
4      118      1
5     1206      1
6     1217      1
7     1227      1
8     1320      1
9      140      1
10     144      1
11    1561      1
12    1610      1
13    1690      1
14     206      1
15    2082      2
16    2093      1
17     212      1
18    2175      1
19     220      2
20    2219      1
21    2226      1
22    2286      1
23     245      1
24     270      1
25    3063      1
26    3146      1
27    3165      1
28    3262      1
29    3282      2
30    3350      1
31     344      1
32    3544      1
33    3724      1
34    3733      1
35    4212      1
36    4237      1
37    4256      1
38    4263      1
39    4268      2
40    4355      1
41    4406      1
42    4550      2
43     458      1
44    4688      1
45     549      1
46     564      1
47     568      1
48     651      1
49     656      1
50     670      1
51     787      1
52     803      1

In [57]:
# foram encontradas agencias de POC nos eventos abaixo filtrados
df_l = df_feat.where(col('label') == 1.0).groupBy('nr_idef_secu').count().orderBy('nr_idef_secu')
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
df_l.toPandas()

nr_idef_secu  count
0           105      1
1          1053      1
2          1074      1
3           112      2
4          1206      6
5          1217     10
6          1227      1
7          1320      1
8           140      4
9          1523      1
10         1530      1
11          154      1
12          156      1
13         1561      1
14         1640      2
15           17      6
16          177      3
17         2082      5
18         2092      4
19         2174      1
20         2175      1
21          220      1
22         2226      3
23         2286      4
24          238      4
25           24      7
26           26      3
27          262      2
28         3063      2
29         3146      2
..          ...    ...
38          344      1
39         3454      8
40         3544     12
41         3618      3
42         3650      1
43         3733      4
44          389      3
45         3912      1
46         3982     11
47         4212      1
48         4237      1
49         4256      1
50         4263      2
51          436      5
52         4406      3
53         4428      5
54         4433      4
55         4550      3
56         4688      1
57         4728      1
58         4776      1
59          568      1
60           65      1
61          652      1
62          670      3
63          678      6
64          690      2
65          734      3
66           86      1
67          990      1

[68 rows x 2 columns]

In [58]:
# checar se selecionou realmente somente agencias com POC do log geral
df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

nr_idef_secu  count
0           3650     11
1           4776      9
2           3454     25
3           1207      1
4            154     10
5            112     41
6           4355      7
7            652     17
8           3733     55
9            670     38
10           389     28
11          1530     16
12          3544    110
13          3063     24
14          1227     28
15          2014      3
16           787     23
17          1074     18
18           644     12
19           188     12
20           568     13
21           458      5
22          3310     17
23           656     33
24          4256     15
25          3350     65
26          3618     19
27          3724     10
28          1610      3
29          4237     10
..           ...    ...
78           803      2
79           105     27
80          3265     47
81          4433     52
82          2219      2
83          1690      8
84          4728     15
85          2047      7
86            65     50
87           734     18
88          4428     34
89          3055     45
90          1525     11
91          2286     46
92           690     34
93          2093     14
94            21     20
95           651      4
96          2226     34
97           151      8
98           262     25
99          3416     23
100         4688      5
101         3371     40
102         3396     10
103         1717      1
104         4268      2
105         3974      5
106          990     14
107         3412     40

[108 rows x 2 columns]

In [67]:
# checar quantas agencias reportadas como POC cairam no log com nova marcação (deveriam aparecer todos )
df_j = df_l \
    .join(df_p, (df_l['nr_idef_secu'] == df_p['agencia']), 'inner')
    
    

In [68]:
df_j.limit(10).toPandas()

nr_idef_secu  count agencia  count
0          112      2     112      2
1         3733      4    3733      1
2          670      3     670      1
3         3544     12    3544      1
4         1227      1    1227      1
5         3063      2    3063      1
6         1074      1    1074      2
7          568      1     568      1
8         4256      1    4256      1
9         3350      2    3350      1